In [1]:
import pandas as pd
import glob
import os
import numpy as np
from matplotlib import pyplot
import seaborn as sns

In [2]:
RESULT_DIR = "./results/results_paper"

In [3]:
def get_latest_results_file(result_dir):    
    list_of_files = glob.glob(result_dir + '/*.csv') 
    latest_file = None
    if list_of_files:
        latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file

In [4]:
results_file = get_latest_results_file(RESULT_DIR)
df = pd.read_csv(results_file)
display(df)

,dataset,optimizer,iteration,variation,runtime
0,Coffee,ssg-1000-1,28_770774145339165431,NaN,785.564273
1,Coffee,ssg-1000-1,4_1685156602300009255,NaN,833.992671
2,Coffee,ssg-1000-1,11_524812844449860411,NaN,853.466319
3,Coffee,adam-1000-1,10_356744795562904667,0.679671,1141.373892
4,Coffee,ssg-1000-10,5_1772159308138482820,0.782782,1182.605506
...,...,...,...,...,...
259,FaceFour,adam-1000-10,22_141371358402523793,74.479456,4079.912006
260,FaceFour,adam-1000-10,24_1367742988523877011,74.479456,3895.455682
261,OliveOil,adam-1000-1,20_2212028830434931373,0.030953,2551.131072
262,FaceFour,adam-1000-10,21_1680389669637796563,82.974364,4417.447519


In [5]:
filter_func = lambda x: x['iteration'].count() != 30 or x['variation'].count() != 30 or x['runtime'].count() != 30
df.groupby(['dataset', 'optimizer']).filter(filter_func)

,dataset,optimizer,iteration,variation,runtime
0,Coffee,ssg-1000-1,28_770774145339165431,NaN,785.564273
1,Coffee,ssg-1000-1,4_1685156602300009255,NaN,833.992671
2,Coffee,ssg-1000-1,11_524812844449860411,NaN,853.466319
9,Coffee,ssg-1000-1,23_381132513222790477,NaN,1383.412142
10,Coffee,ssg-1000-1,13_2303319081865907568,NaN,1419.238617
...,...,...,...,...,...
252,OliveOil,ssg-1000-1,24_1420141314657169739,NaN,3091.675664
253,OliveOil,ssg-1000-1,15_865162270781419870,NaN,3187.067898
254,OliveOil,ssg-1000-1,23_1413915879820702052,NaN,3118.798364
256,OliveOil,ssg-1000-1,28_2303558225644958105,NaN,3115.080040


In [6]:
unfinshed_datasets = df.groupby(['dataset', 'optimizer']).filter(filter_func)['dataset'].unique()
unfinshed_datasets

array(['Coffee', 'FaceFour', 'OliveOil'], dtype=object)

In [7]:
df_filtered = df[~df.dataset.isin(unfinshed_datasets)]

In [8]:
df_grouped = df_filtered.groupby(['dataset', 'optimizer']).agg(['mean', 'std'])
display(df_grouped)

Empty DataFrame
Columns: [(variation, mean), (variation, std), (runtime, mean), (runtime, std)]
Index: []

In [9]:
df_variation = df_filtered.drop(columns=['runtime'])
df_variation.groupby(['dataset', 'optimizer']).agg(['mean', 'std'])

Empty DataFrame
Columns: [(variation, mean), (variation, std)]
Index: []

In [10]:
df_plot = df_variation.groupby(['dataset', 'optimizer']).agg(['mean', 'std'])
df_plot.columns = df_plot.columns.to_flat_index()
df_plot.reset_index(inplace=True)

df_plot = pd.melt(df_plot, id_vars=['dataset', 'optimizer'])
display(df_plot)

,dataset,optimizer,variable,value


In [11]:
with sns.plotting_context('notebook', font_scale=2):
    sns.catplot(x='dataset', y='value', hue='optimizer', col='variable', data=df_plot, height=10, s=20, kind='swarm')

ValueError: min() arg is an empty sequence

In [ ]:
REFERENCE_FILE = "reference/5-sets_reference-results.csv"
df_r = pd.read_csv(REFERENCE_FILE)
df_r.groupby(['dataset']).mean()

In [ ]:
df_sign = df.copy()
df_sign['variation_sign'] = np.sign(df.variation)
df_sign.groupby(['dataset', 'optimizer']).variation_sign.value_counts().unstack()